In [2]:
import pandas as pd
import os
import numpy as np 

pd.set_option('display.max_columns', None)

In [3]:
def sort_path(dir_list):
    return sorted(dir_list, key=lambda x : x.split('_')[1])

def add_path(dir_list, path):
    return [path + '/' + i for i in dir_list]

In [4]:
path = "HIKARI-CICFlowMeter/labeled"
files = os.listdir(path)
files = add_path(files, path)

# selcting only the ones mentioned on the paper

In [5]:
files

['HIKARI-CICFlowMeter/labeled/Sunday_2021-03-28.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Saturday_2021-04-10_1628.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Wednesday_2021-04-07.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Tuesday_2021-03-30.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Probing.csv',
 'HIKARI-CICFlowMeter/labeled/Monday_2021-03-29.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Saturday_2021-05-01_0520.anonymized.pcap_ISCX.csv',
 'HIKARI-CICFlowMeter/labeled/Bruteforce.csv',
 'HIKARI-CICFlowMeter/labeled/Bruteforce-XML.csv']

In [6]:
ds = pd.concat((pd.read_csv(file) for file in files), ignore_index=True)
ds.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,52.70.104.90-98.121.8.240-443-27160-6,98.121.8.240,27160,52.70.104.90,443,6,28/03/2021 06:13:10,103850,9,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,144.439095,7417.857143,10343.423677,25101.0,24.0,103850.0,12981.250000,11418.317424,26173.0,89.0,77976.0,15595.200000,20064.580153,39959.0,24.0,0,0,0,0,296,200,86.663457,57.775638,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,296,0,0,0,0,0,0,9,0,6,0,29200,261,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,52.70.104.90-98.121.8.240-443-27160-6,52.70.104.90,443,98.121.8.240,27160,6,28/03/2021 06:13:10,24977,4,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,240.221003,4995.400000,10609.213133,23961.0,18.0,24977.0,8325.666667,13581.451555,24000.0,50.0,23993.0,23993.000000,0.000000,23993.0,23993.0,0,0,0,0,128,40,160.147336,80.073668,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,128,0,0,0,0,0,0,4,0,2,0,265,0,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,52.70.104.60-98.121.8.240-443-30398-6,98.121.8.240,30398,52.70.104.60,443,6,28/03/2021 06:13:09,1086757,10,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.642871,67922.312500,165480.725561,648569.0,20.0,1086757.0,120750.777778,213671.405183,648569.0,38.0,412655.0,68775.833333,107497.527327,271557.0,82.0,0,0,0,0,328,232,9.201689,6.441182,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,328,0,0,0,0,0,0,10,0,7,0,29200,265,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,52.70.104.60-98.121.8.240-443-30394-6,98.121.8.240,30394,52.70.104.60,443,6,28/03/2021 06:13:09,1254120,71,69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.632061,9022.446043,59577.494931,688656.0,17.0,1254120.0,17916.000000,84116.788584,688656.0,30.0,540955.0,7955.220588,25087.237688,138880.0,37.0,0,0,0,0,3172,2216,56.613402,55.018659,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,3172,0,0,0,0,0,0,71,0,69,0,29200,277,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,36.168.45.73-98.121.8.240-80-31870-6,98.121.8.240,31870,36.168.45.73,80,6,28/03/2021 06:13:09,1428964,5,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.598462,204137.714286,529072.482220,1403894.0,13.0,1428964.0,357241.000000,697789.115906,1403894.0,802.0,13018.0,6509.000000,9178.246020,12999.0,19.0,0,0,0,0,168,104,3.499038,2.099423,0.0,0.0,0.0,0.0,0.0,0,0,0,1,0,0,0,0,0.0,0.0,0.0,0.0,168,0,0,0,0,0,0,5,0,3,0,29200,261,0,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [7]:
# removing empty space in the column name
ds.columns = [i.strip() for i in ds.columns]

In [8]:
ds['Timestamp'] = pd.to_datetime(ds['Timestamp'], format='%d/%m/%Y %H:%M:%S')

In [9]:
ds.sort_values('Timestamp', inplace=True, ignore_index=True)

In [10]:
ds['Label'] = np.where(ds['Label'] == 'BENIGN', 'Background', ds['Label'])

In [11]:
# Function to remove the latest number after the hyphen
def remove_latest_number(text):
    x = text.split('-')
    return x[0] + '-' + x[1] + '-' + x[2] + '-' + x[3]

In [12]:
ds['Flow ID'] = ds['Flow ID'].apply(remove_latest_number)

In [13]:
# loading the connection that has perfomed the attack
miner_id = pd.read_csv('miner_id.csv', index_col=0)
miner_id

,con_id
23387,103.255.15.255-103.255.15.42-137-137
23388,103.255.15.255-103.255.15.42-137-137
23389,103.255.15.255-103.255.15.42-137-137
23390,103.255.15.255-103.255.15.42-137-137
23391,103.255.15.255-103.255.15.42-137-137
...,...
26661,103.255.15.255-103.255.15.42-138-138
26662,103.255.15.255-103.255.15.42-138-138
26663,103.255.15.255-103.255.15.42-138-138
26664,103.255.15.255-103.255.15.42-138-138


In [ ]:
# creating a function to label the connection
con_id_list = pd.unique(miner_id['con_id']).to_list()

def label_connection(flow_id):
    if flow_id in con_id_list:
        return 'XMRIGCC CryptoMiner'
    else:
        return 'Background'

file9['Label'] = file9['Flow ID'].apply(label_connection)

In [18]:
con_id_list = miner_id['con_id'].to_list()

np.where(ds['Flow ID'] in con_id_list, 'XMRIGCC CryptoMiner', ds['Label'])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [63]:
ds.to_csv('HIKARI-2021-CICFlowMeter-Labeled.csv')

In [65]:
ds['Label'].value_counts()

Label
Benign            348606
Background        188936
Probing            64112
Bruteforce-XML      8221
Bruteforce          1127
Name: count, dtype: int64